# TF-IDF

We'll rely on term frequency times inverted document frequency ([TF-IDF](https://web.stanford.edu/~jurafsky/slp3/11.pdf)), a powerful implementation of the [bag-of-words model](https://web.stanford.edu/~jurafsky/slp3/B.pdf), to measure meaningful similarity between documents while disregarding word order. Let's start by generating a matrix for the separate constituent parts of _Stjórn_.

In [ ]:
import os,glob,json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
def normalize(target):
    # This dict limits orthographical variation beyond the rule sets
    # of stjorn-extract.ipynb and menota-extract.ipynb:
    matrix = {
        'j': 'i',
        'v': 'u',
        # Use of d for ð in AM 226 rather spoils the results,
        # so instead of normalizing ð to þ we will normalize
        # both to d (resulting in strange forms of course):
        #'ð': 'þ',
        'ð': 'd',
        'þ': 'd',
        'á': 'a',
        'ǽ': 'æ',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ý': 'y',
        'ǿ': 'ø',
        'k': 'c', # rather than vice versa, because of Latin (e.g. Lucifer)
        '[': '',
        ']': ''
        }
    for k,v in matrix.items():
        target = target.replace(k, v)
    return target

titles = ['prologue', 'introduction', 'gn', 'ex', 'lv', 'nm', 'dt', 'ios', 'idc', 'rt', '1sm', '2sm', '3rg', '4rg']
tokens = []
for title in titles:
    with open(f"nlp/{title}.txt") as raw:
        document = raw.read().replace('\n', ' ')
        tokens.extend(document.split())

work_indices = {
    'stjorn1': (650,124417),
    'stjorn2': (124417,147678),
    'stjorn3': (147678,156943,160719),
    'stjorn4': (156943,160719)
}

stjorn = dict()
for _work, _range in work_indices.items():
    if len(_range) == 2:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]]))
    else:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]] + tokens[_range[2]:]))

menota = dict()
for text in glob.glob('../menota/dipl/*txt'):
    ref = os.path.basename(text).replace('.txt', '')
    with open(text) as doc:
        # We'll subject Menota to the same normalization standard as Stjórn:
        menota[ref] = normalize(doc.read().replace('\n', ''))

Note the arguments passed to the vectorizer class below. `min_df` sets a cutoff for the minimum number of documents in which a term has to appear in order to be included in the model. Since terms exclusive to single compositions are among the things that interest us, we'll leave this at `1`. `max_df` sets a cutoff point above which relative document frequency a term is ignored: in other words, a value of `0.8` ignores words occurring in over 80 percent of documents. Changing this value massively changes the document similarity scores downstream, and drastic changes in the setting have a pronounced effect on the document similarity rankings as well. A strict threshold is at any rate required to gain an insight into the relevance of individual terms, as leaving `max_df` at its default of `1.0` would lead the model to conclude that "oc" is the most meaningful term in many of our documents, while "æigi" is the top-ranking term for _Stjórn III_ as well as the _Norwegian Homily Book_ with a value as low as `0.4`. We can therefore leave the score high in the _Stjórn_-internal comparison, but may want to set a lower threshold for the larger corpus.

In [7]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.8)
model = vectorizer.fit_transform(stjorn.values())
df = pd.DataFrame(cosine_similarity(model), stjorn.keys(), stjorn.keys())
df

,stjorn1,stjorn2,stjorn3,stjorn4
stjorn1,1.000000,0.059320,0.682042,0.425337
stjorn2,0.059320,1.000000,0.083006,0.025865
stjorn3,0.682042,0.083006,1.000000,0.479767
stjorn4,0.425337,0.025865,0.479767,1.000000


After eliminating such variation as vowel length marks and the þ/ð distinction, these are now all pretty similar to one another, with the biggest difference between _Stjórn II_ and _III_.

Now let's first add _Konungs skuggsjá_ from Menota, as well as Unger's own edition of the _Norwegian Homily Book_. Fingers crossed that we have got the normalization standard of the former to approach Unger's methods reasonably well.

In [8]:
# We want only those parts of Unger's NHB matched in Menota:
nhb_titles = ['alcuin', 'hom', 'olafr', 'visio', 'paternoster', 'anhang1']
nhb = ''
for title in nhb_titles:
    filepath = f'../nhb/nlp/{title}.txt'
    with open(filepath) as doc:
        nhb = nhb + normalize(doc.read().replace('\n', ''))
stjorn_plus = []
for v in stjorn.values():
    stjorn_plus.append(v)
stjorn_plus.extend([menota['nks235g_konungs_skuggsja'], nhb])
model = vectorizer.fit_transform(stjorn_plus)
df = pd.DataFrame(cosine_similarity(model), list(stjorn.keys()) + ['ks', 'nhb'], list(stjorn.keys()) + ['ks', 'nhb'])
df

,stjorn1,stjorn2,stjorn3,stjorn4,ks,nhb
stjorn1,1.000000,0.245597,0.320679,0.343395,0.007072,0.048417
stjorn2,0.245597,1.000000,0.229203,0.301802,0.084316,0.034099
stjorn3,0.320679,0.229203,1.000000,0.332619,0.014582,0.210626
stjorn4,0.343395,0.301802,0.332619,1.000000,0.001074,0.072545
ks,0.007072,0.084316,0.014582,0.001074,1.000000,0.055996
nhb,0.048417,0.034099,0.210626,0.072545,0.055996,1.000000


_Stjórn III_ and _Konungs skuggsjá_ share material cognate within the vernacular, but not so much, or with insufficient spelling agreement, to stand out in this matrix. In fact, the _Norwegian Homily Book_ has a higher match with _Stjórn III_ than _Konungs skuggsjá_ does, which may be explained at least in part by the closer subject match for those parts of _Stjórn III_ not reflected in _Konungs skuggsjá_.

Next, let's model all of Menota along with Stjórn. Perhaps we'll leave Unger's _Homily Book_ in alongside the Menota edition as a proof of method for now.

In [14]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.6)
corpus = []
titles = []
for k,v in stjorn.items():
    titles.append(k)
    corpus.append(v)
titles.append('nhb')
corpus.append(nhb)
for k,v in menota.items():
    titles.append(k)
    corpus.append(v)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df.sort_values(by=['stjorn3'], ascending=False)

,stjorn1,stjorn2,stjorn3,stjorn4,nhb,nraNorrFragm75_kross_saga,am132_egils_saga,am162btheta_njals_saga,nraNorrFragm64_barlaams_saga,nraNorrFragm81A_benedikts_regla,am1056IX_konungs_skuggsja_fragment,am78_kristinrettir,am63_heimskringla3,dg4-7_strengleikar,am132_droplaugasona_saga,am132_kormaks_saga,nraNorrFragm72x76_dialogar,nraNorrFragm53_haralds_saga_hardrada,am132_finnboga_saga,nraNorrFragm70_agotu_saga,nraNorrFragm62_karlamagnuss_saga,nraNorrFragm60A_stjorn,am132_fostbraedra_saga,lbsFragm82_olafs_saga_helga,nraNorrFragm58B_konungs_skuggsja,nraNorrFragm60C_stjorn,holmPerg30_landslog,am619_norwegian_homily_book,nraNorrFragm57_jons_saga_helga,nraNorrFragm69_nikulass_saga,am56_landslog,wolfAug9-10_egils_saga,nraNorrFragm66_thomass_saga,holmPerg17_thomass_saga,am383I_thorlaks_saga,holmPerg4_thidreks_saga,am132_njals_saga,am36_heimskringla2,am544_voluspa,am162bkappa_njals_saga,am305_landslog,nraNorrFragm58C_konungs_skuggsja,am132_olkofra_thattr,konungs_skuggsja_am243ba,nraNorrFragm54_sverris_saga,nraNorrFragm55B_hakonar_saga,nraNorrFragm79_mariu_saga,gks2365_voluspa,am243balpha_konungs_skuggsja,nraNorrFragm51_fagrskinna,am132_viga-glums_saga,am279a_gragas,am677_gregory,am132_laxdoela_saga,am302_landslog,am178_thidreks_saga,nraNorrFragm81B_benedikts_regla,am132_bandamanna_saga,nraNorrFragm71_gregors_saga_pafa,am655_laeknisbok,am519a_alexanders_saga,holmPerg34_landslog,am162balpha_njals_saga,am113_islendingabok,nraNorrFragm7_landslog,nraNorrFragm67_thomass_saga,nraNorrFragm56_thorgils_saga,nks235g_konungs_skuggsja,am132_hallfredar_saga,am35_heimskringla1,am242_codex_wormianus,nraNorrFragm78_mariu_saga,konungs_skuggsja_fragment_am1056xi,dg8II_olafs_saga,nraNorrFragm80_pals_saga,nraNorrFragm63_karlamagnuss_saga,nraNorrFragm77_dialogar,am28_codex_runicus,holmPerg34_boejarlog,dg8I_landslog,nraNorrFragm60B_stjorn,nraNorrFragm55A_hakonar_saga,skbA120_marys_complaint,nraNorrFragm59_rimbegla,nraNorrFragm65_floress_saga,nraNorrFragm52_olafs_saga_helga_hin_elzta,holmPerg6_barlaams_saga,nraNorrFragm68_brendanuss_saga,nraNorrFragm61_karlamagnuss_saga,nraNorrFragm58A_konungs_skuggsja
stjorn3,0.312301,0.274925,1.000000,0.335113,0.255194,0.029514,0.233396,0.096043,0.110605,0.035001,0.017462,0.110909,0.111568,0.220797,0.131979,0.077022,0.056118,0.111631,0.092018,0.030573,0.194054,0.047320,0.130464,0.197158,0.173695,0.233327,0.091197,0.250506,0.125361,0.069946,0.112693,0.270733,0.137239,0.179653,0.048310,0.284971,0.173084,0.127200,0.068341,0.059824,0.042693,0.039952,0.103513,0.164425,0.180277,0.100596,0.112160,0.038482,0.164425,0.039807,0.134650,0.043880,0.067121,0.184134,0.044956,0.030339,0.109532,0.097506,0.077531,0.027792,0.172252,0.059283,0.056386,0.026999,0.141413,0.180012,0.094936,0.065172,0.135512,0.123365,0.237395,0.109988,0.017462,0.169611,0.107091,0.176795,0.112791,0.028241,0.041300,0.200600,0.158172,0.189536,0.010052,0.032681,0.032716,0.125467,0.141543,0.078644,0.126338,0.077348
stjorn4,0.334208,0.320630,0.335113,1.000000,0.094347,0.012554,0.122759,0.045897,0.035702,0.015217,0.011482,0.040990,0.052128,0.080421,0.089178,0.045586,0.023553,0.034345,0.053268,0.011128,0.097666,0.050217,0.084528,0.053657,0.043357,0.051158,0.033748,0.089026,0.050085,0.037556,0.041142,0.139042,0.034169,0.103573,0.021960,0.088559,0.117359,0.057418,0.037778,0.033918,0.016733,0.015181,0.074796,0.054163,0.100377,0.027084,0.028593,0.019015,0.054163,0.015462,0.085139,0.013238,0.026303,0.102419,0.016362,0.012874,0.032252,0.055506,0.031931,0.009776,0.050662,0.034566,0.025938,0.011914,0.036866,0.064411,0.031581,0.022767,0.075785,0.058358,0.093268,0.040878,0.011482,0.054284,0.054259,0.047925,0.050480,0.005588,0.022548,0.048840,0.016721,0.078266,0.002868,0.038535,0.012178,0.032370,0.059039,0.039102,0.036768,0.027549
stjorn1,1.000000,0.312637,0.312301,0.334208,0.093017,0.019810,0.103857,0.060350,0.043895,0.020408,0.039041,0.027957,0.075876,0.102881,0.067388,0.042794,0.045419,0.036157,0.057299,0.023214,0.110840,0.190631,0.070986,0.043890,0.099508,0.071635,0.024864,0.088199,

The score for the two editions of the _Norwegian Homily Book_ may serve as our proof of method: with any `max_df` setting, and whether or not we normalize &lt;þ&gt; and &lt;ð&gt; to &lt;d&gt;, these come to a similarity in the range 0.93~0.97. As this compares an edition of Unger's with a Menota transcription, as does our comparison of _Stjórn_ with the remainder of the Menota corpus, we may be confident that the scores give a fair indication of lexical similarity to the extent the TF-IDF measure can provide one.

Let's take the constituent parts of _Stjórn_ one at a time and rank the Menota material by similarity. At this point, having demonstrated the validity of our method at least for editions of the same manuscript, we may remove Unger's _Homily Book_ from the data set:

In [10]:
nhb_index = titles.index('nhb')
corpus.pop(nhb_index)
titles.pop(nhb_index)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

def rank(subject):
    sorted_df = df.sort_values([subject], ascending=False)
    print(sorted_df[subject][:10].to_string()) # Limiting output for brevity

In [15]:
for i in work_indices.keys():
    print(f"{i}:")
    print("--------------------------------------------")
    rank(i)
    print('')

stjorn1:
--------------------------------------------
stjorn1                        1.000000
holmPerg17_thomass_saga        0.366871
stjorn4                        0.334208
stjorn2                        0.312637
stjorn3                        0.312301
nraNorrFragm60A_stjorn         0.190631
am242_codex_wormianus          0.160517
holmPerg6_barlaams_saga        0.135757
nraNorrFragm66_thomass_saga    0.134836
wolfAug9-10_egils_saga         0.122039

stjorn2:
--------------------------------------------
stjorn2                     1.000000
stjorn4                     0.320630
stjorn1                     0.312637
stjorn3                     0.274925
holmPerg34_landslog         0.198458
holmPerg34_boejarlog        0.175979
holmPerg6_barlaams_saga     0.171602
am132_njals_saga            0.165271
nraNorrFragm80_pals_saga    0.149032
am132_laxdoela_saga         0.143239

stjorn3:
--------------------------------------------
stjorn3                        1.000000
stjorn4                   

These are complex results. The redaction of _Thómass saga erkibyskups_ transmitted in Holm. Perg. 17 was "uden Tvivl tilbleven i Norge" as judged by its language and syntax, but probably later in the second half of the 13th century (Unger iii); "Oversættelsens Stil minder undertiden om Kongespeilet" (ibid.). The fact of its Norwegian origin alone does not explain an affinity with _Stjórn I_ against _Stjórn III_ (both of whose Norwegian ancestry is thought to be at two generations' remove at least), while the date of _Thomass saga_ would rather associate it with the latter. As for the similarity of style between _Thómass saga_ and _Konungs skuggsjá_ as observed by Unger, he cannot have meant a matter of style reflected in the cosine similarity of their TF-IDF within this corpus, as that is decidedly low:

In [21]:

df['holmPerg17_thomass_saga'].loc['nks235g_konungs_skuggsja']

0.045528920560418525

Since TF-IDF is a reflection of unusual word forms, our next step is to investigate which forms are conspicuously associated with the constituent parts of _Stjórn_. The outcome of this query is even more greatly affected by our `max_df` setting, and this is the test that prompted the equation between the characters &lt;þðd&gt; in preprocessing above, as Unger edited _Stjórn I_ from AM 226, which tends to use &lt;d&gt; for &lt;ð&gt;. This, as well as other normalization strategies in this notebook and others it depends on, should be kept in mind when consulting the below rankings of the most striking forms in each constituent part of _Stjórn_. It should also be remembered that commonly spelled function words have been eliminated using the `max_df` setting:

In [12]:
scores = pd.DataFrame(model.toarray(), titles, vectorizer.get_feature_names_out())
for i in work_indices.keys():
    print(f"{i}:")
    print("------------------------")
    print(scores.loc[i].sort_values(ascending=False)[:20])
    print('')

stjorn1:
------------------------
medr          0.547546
aa            0.371156
edr           0.248992
di            0.206674
diat          0.197833
eptir         0.146892
man           0.106545
ders          0.102619
scylld        0.101959
deirra        0.100019
tima          0.096744
uaru          0.093733
scolastica    0.091207
sealfs        0.084231
ioseph        0.083724
hefir         0.082703
taladi        0.078276
iacob         0.074836
sagdi         0.073801
eru           0.073116
Name: stjorn1, dtype: float64

stjorn2:
------------------------
aa          0.452333
dier        0.303516
moyses      0.263091
firir       0.230147
uit         0.225063
suo         0.206921
mællti      0.195620
scaltu      0.171302
aaron       0.166038
yfir        0.158607
drottinn    0.153031
cyni        0.118664
brutt       0.114661
balaam      0.107145
moysen      0.103471
munu        0.100776
deirra      0.092715
duiat       0.092427
drottins    0.084714
sculu       0.079317
Name: stjorn2, dtype:

_Stjórn I_ stood out for its form "medr" *before* the conflation of &lt;þðd&gt;, but it continues to be its most remarkable form after, occurring as it does in 16 out of 85 Menota items. What is also striking is that the exact form "medr" so frequently found in AM 226 occurs three times in the Holm. Perg. 17 manuscript of _Thómass saga erkibyskups_, while the AM 132 text of _Laxdœla saga_ and the Norr. Fragm. 58B fragment of _Konungs skuggsjá_ have it once each. "Medr" is thus not only a form that sets (the AM 226 text of) _Stjórn I_ apart, it is also one of the forms associating it with _Thómass saga_.